# Coursera Project
## Identify new business opportunities

#### Step 1: Data Gathering

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Scrape the Wikipedia Page
website_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

content = requests.get(website_url).text
soup = BeautifulSoup(content, 'lxml')
codes_html = soup.find('table', {'class':'wikitable sortable'})

table_rows = codes_html.find_all('tr')

df_rows = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text.strip() for i in td]
    if row != []:
        df_rows += [row]

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(df_rows, columns=['Postcode', 'Borough', 'Neighborhood'])

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.    
df = df[df['Borough'] != 'Not assigned']

#More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = df.groupby(by=['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df['Neighborhood'] = df.apply(lambda x: x['Borough'] if x['Neighborhood']=='Not assigned' else x['Neighborhood'], axis=1)

# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe
df.shape

(103, 3)

In [4]:
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [2]:
# Use the Geocoder package or the csv file to create the following dataframe:
df2 = pd.read_csv('Geospatial_Coordinates.csv')
df3 = pd.merge(df, df2, on='Postcode')
df3.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [3]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [4]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

c:\python27\lib\site-packages\folium\__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


In [5]:
CLIENT_ID = 'XXX' # your Foursquare ID
CLIENT_SECRET = 'XXX' # your Foursquare Secret
VERSION = '20181218' # Foursquare API version

def getNearbyVenues(names, latitudes, longitudes, LIMIT=200, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
venues = getNearbyVenues(names=df3['Neighborhood'],
                                  latitudes=df3['Latitude'],
                                  longitudes=df3['Longitude'],
                                  LIMIT=200)

print('The "venues" dataframe has {} venues and {} unique venue types.'.format(
      len(venues['Venue Category']),
      len(venues['Venue Category'].unique())))

venues.to_csv('venues.csv', sep=',', encoding='UTF8')
venues.head()

The "venues" dataframe has 4844 venues and 327 unique venue types.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge, Malvern",43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
3,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
4,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant


#### Step 2: Preparing the data

In [7]:
# one hot encoding
venue_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

#column lists before adding neighborhood
column_names = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude'] + list(venue_onehot.columns)

# add neighborhood column back to dataframe
venue_onehot['Neighborhood'] = venues['Neighborhood']
venue_onehot['Neighborhood Latitude'] = venues['Neighborhood Latitude']
venue_onehot['Neighborhood Longitude'] = venues['Neighborhood Longitude']

# move neighborhood column to the first column
venue_onehot = venue_onehot[column_names]

venue_onehot.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Rouge, Malvern",43.806686,-79.194353,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",43.806686,-79.194353,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge, Malvern",43.806686,-79.194353,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge, Malvern",43.806686,-79.194353,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Rouge, Malvern",43.806686,-79.194353,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
venue_onehot.reset_index()
venues_freq = venue_onehot.groupby(['Neighborhood Latitude', 'Neighborhood Longitude']).mean().reset_index()
venues_freq.head()

,Neighborhood Latitude,Neighborhood Longitude,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,43.602414,-79.543484,0.0,0.000000,0.0,0.0000,0.0000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1,43.605647,-79.501321,0.0,0.000000,0.0,0.0000,0.0000,0.050000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
2,43.628841,-79.520999,0.0,0.017241,0.0,0.0000,0.0000,0.034483,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.034483,0.0
3,43.628947,-79.394420,0.0,0.000000,0.0,0.0625,0.0625,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
4,43.636258,-79.498509,0.0,0.000000,0.0,0.0000,0.0000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


In [62]:
res_cols = [col for col in venues_freq.columns if 'restaurant' in col.lower()]
res_df = venues_freq[['Neighborhood Latitude','Neighborhood Longitude'] + spike_cols]
res_df.head()

,Neighborhood Latitude,Neighborhood Longitude,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,...,Sri Lankan Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,43.602414,-79.543484,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,43.605647,-79.501321,0.0,0.050000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,43.628841,-79.520999,0.0,0.034483,0.0,0.017241,0.0,0.0,0.0,0.0,...,0.0,0.034483,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0
3,43.628947,-79.394420,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,43.636258,-79.498509,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


#### Step 4: Model training

In [63]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
#venues_clustering = venues_freq.drop(['Neighborhood Latitude', 'Neighborhood Longitude'], 1)
venues_clustering = res_df.drop(['Neighborhood Latitude', 'Neighborhood Longitude'], 1)

for n_cluster in range(2, 10):
    kmeans = KMeans(n_clusters=n_cluster).fit(venues_clustering)
    label = kmeans.labels_
    sil_coeff = silhouette_score(venues_clustering, label, metric='euclidean')
    print("For n_clusters={}, The Silhouette Coefficient is {}".format(n_cluster, sil_coeff))

For n_clusters=2, The Silhouette Coefficient is 0.755766447754
For n_clusters=3, The Silhouette Coefficient is 0.344986005479
For n_clusters=4, The Silhouette Coefficient is 0.0950850090063
For n_clusters=5, The Silhouette Coefficient is 0.0962752875768
For n_clusters=6, The Silhouette Coefficient is 0.105353802099
For n_clusters=7, The Silhouette Coefficient is 0.0924335909059
For n_clusters=8, The Silhouette Coefficient is 0.111104959923
For n_clusters=9, The Silhouette Coefficient is 0.11232141954


In [64]:
from sklearn.cluster import KMeans
kclusters = 2

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_clustering)

# clustered_df = venues_freq
clustered_df = spike_df
clustered_df['Cluster'] = kmeans.labels_

clustered_df.sort_values(['Cluster'], inplace=True)
clustered_df.head()

c:\python27\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\python27\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,Neighborhood Latitude,Neighborhood Longitude,Afghan Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Cantonese Restaurant,...,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Cluster
0,43.602414,-79.543484,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
72,43.733283,-79.419750,0.0,0.025641,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.025641,0.0,0.0,0.025641,0.0,0.0,0.0,0.0,0.0,0
71,43.728496,-79.495697,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.5,0
69,43.727929,-79.262029,0.0,0.000000,0.0,0.040000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
68,43.725900,-79.340923,0.0,0.045455,0.0,0.045455,0.0,0.0,0.0,0.0,...,0.022727,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0


In [72]:
new_df = pd.merge(df3, clustered_df,  how='right', left_on=['Latitude','Longitude'], right_on = ['Neighborhood Latitude','Neighborhood Longitude'])
new_df.to_csv('clustered.csv')
new_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Neighborhood Latitude,Neighborhood Longitude,Afghan Restaurant,American Restaurant,Argentinian Restaurant,...,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Cluster
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,43.806686,-79.194353,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,43.784535,-79.160497,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,43.763573,-79.188711,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,43.770992,-79.216917,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,43.773136,-79.239476,0.0,0.0,0.0,...,0.0,0.0,0.0,0.034483,0.0,0.0,0.0,0.0,0.0,0


#### Step 4: Observation

In [73]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for nei, lat, lon, cluster in zip(new_df['Neighborhood'], new_df['Neighborhood Latitude'], new_df['Neighborhood Longitude'], clustered_df['Cluster']):
    label = folium.Popup('{} Cluster {} {} {}'.format(nei, cluster, lat, lon), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [74]:
new_df[new_df['Cluster'] == 1].to_csv('temp.csv')
new_df[new_df['Cluster'] == 1] # Single blue dot 

,Postcode,Borough,Neighborhood,Latitude,Longitude,Neighborhood Latitude,Neighborhood Longitude,Afghan Restaurant,American Restaurant,Argentinian Restaurant,...,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Cluster
43,M4N,Central Toronto,Lawrence Park,43.72802,-79.38879,43.72802,-79.38879,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


**Observation**
There is one neighborhod that doesn't have any good restaurants yet (probably), and is a potential business opportunity for restauranteurs